#Conexión API-Facebook y scrapeo





Este notebook tiene como propósito conectar con la API de Meta (Facebook)mediante solicitudes HTTP para extraer datos sobre anuncios publicitarios activos, archivados o relacionados con temas sociales, elecciones o política en Colombia. La información recolectada permite estudiar variables como:


*  Identidad del anunciante, fechas de inicio y fin del anuncio,gasto estimado, alcance demográfico (edad, sexo, ubicación), contenidos promocionados.


También automatiza la descarga de contenido multimedia de anuncios en Facebook de un candidato. Extrae videos, imágenes, carruseles y artículos desde un enlace proporcionado, guardándolos en una carpeta de Google Drive. Usa Selenium para manejar la interacción con la página. Los archivos se guardan con la función save_file, que descarga los enlaces y los guarda localmente. Además, check_file evita descargar anuncios duplicados.

El análisis está pensado para contribuir a investigaciones sobre transparencia, desinformación y segmentación en campañas políticas, usando herramientas de análisis de datos como pandas y herramientas como selenium.

Requisitos: Para ejecutar este notebook, es necesario contar con un token de acceso válido de la API de Meta for Developers y permisos adecuados para consultar la biblioteca de anuncios.



##Instalacion de librerias

In [ ]:
import json
import requests
import pandas as pd
from pprint import pprint
import requests
import glob

# drive para almacenar los archivos
from google.colab import drive
drive.mount('/content/drive')

# Selenium
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

# Guardar archivos
import os

# seguimiento de errores
import traceback

import time

#UbicacionCSV

os.chdir("/content/drive/MyDrive/Proyecto_grado/csv_carpetas")

In [ ]:
# Selenium

# Configuración para ejecutar Selenium en Google Colab

%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,122 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,462 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restr

In [ ]:
!pip install chromedriver-autoinstaller

In [ ]:
# correr selenium

# setup chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# set path to chromedriver as per your configuration
chromedriver_autoinstaller.install()

# set up the webdriver
driver = webdriver.Chrome(options=chrome_options)



# login facebook
driver.get('https://www.facebook.com')

input_email = WebDriverWait(driver, 4).until( EC.presence_of_element_located((By.ID, "email")))
input_email.send_keys('')

input_pass = WebDriverWait(driver, 4).until( EC.presence_of_element_located((By.ID, "pass")))
input_pass.send_keys('')

btn_log = WebDriverWait(driver, 4).until( EC.presence_of_element_located((By.XPATH, "//button[@value='1']")))
btn_log.click()


## Credenciales API key



In [ ]:
api_token = "EAAP88z0YeOYBO2VAwMBiFO7kSC3LLOYQ7jAgqJS49tl8MXZBcRink0LMZAAPebGnudkwy5JaWVFLHbzkXqYDAKzvGrtnE2o0ZBo7xJuMkJaV0twmQUmZBu7d49jFVfktixdNsDXbCNFuSUzLUMZAkgXxxYilEuAjFvL2m9xymBv8BUBC1IvtVFlOMMaybWSEwOgWeszI5XZAdGucsavCoFmI9oOljlZCTY5EnnhxUZAy"
version_api = "v19.0"

##Prueba API



In [ ]:
parametros= {"search_terms":'california' ,
             "ad_type":'POLITICAL_AND_ISSUE_ADS',
             "ad_reached_countries":['US'],
             "access_token":api_token,
             "estimated_audience_size_max":1000000}

#URL API Esta es la URL

url_api= "https://graph.facebook.com/v19.0/ads_archive"


# Realizar la solicitud GET
response = requests.get(url_api, params=params)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    response_json = response.json()
    pprint(response_json)
else:
    print("Error:", response.text)

{'data': [{'ad_delivery_start_time': '2023-10-28',
           'ad_delivery_stop_time': '2023-10-29',
           'ad_snapshot_url': 'https://www.facebook.com/ads/archive/render_ad/?id=836514814873364&access_token=EAAP88z0YeOYBO31lGacDGfNB7JeGpW8ZBk198Wgnzb2MRB2ZCXvdk1720C7od21ZBVrBJZBRAkH1eo9LqZBmBejDl0SXgif83hDCjphahRlPbIXbRztZCSJuAmlPU1hkY1yAASkUYoXzRjWOruryAg4WH7Yp6dZCRaSBFLBFNT5ZARmE53E13ZCZCkbqPF3gHgtOc3jmI2f5HzZBk3Pg4IYQ0ZAxLsfhBAAPCgUHzPquVUNp',
           'id': '836514814873364',
           'page_id': '241499431662'},
          {'ad_delivery_start_time': '2023-10-27',
           'ad_delivery_stop_time': '2023-10-29',
           'ad_snapshot_url': 'https://www.facebook.com/ads/archive/render_ad/?id=271341335898055&access_token=EAAP88z0YeOYBO31lGacDGfNB7JeGpW8ZBk198Wgnzb2MRB2ZCXvdk1720C7od21ZBVrBJZBRAkH1eo9LqZBmBejDl0SXgif83hDCjphahRlPbIXbRztZCSJuAmlPU1hkY1yAASkUYoXzRjWOruryAg4WH7Yp6dZCRaSBFLBFNT5ZARmE53E13ZCZCkbqPF3gHgtOc3jmI2f5HzZBk3Pg4IYQ0ZAxLsfhBAAPCgUHzPquVUNp',
           'i

## Prueba parámetros y campos de consulta

In [ ]:
# Parámetros
params = {
    "search_page_ids": '241499431662', # id de la pagina de Fico
    "ad_type": "POLITICAL_AND_ISSUE_ADS",
    "ad_reached_countries": ["CO"],
    "ad_delivery_date_min":"2023-08-01",
    "ad_delivery_date_max": "2023-12-30",
    "access_token": api_token

    }

In [ ]:
# Campos de consulta
query = 'id, spend, currency, page_name, ad_creation_time, ad_delivery_start_time, ad_snapshot_url, impressions, publisher_platforms, ad_creative_bodies, estimated_audience_size, demographic_distribution'

#URL API Esta es la URL
url_api= "https://graph.facebook.com/v19.0/ads_archive"

# Realizar la solicitud GET
response = requests.get(url_api, params=params)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:

    response_json = response.json()

    for i in response_json['data']:
      print('')
      pprint(i)

    print("")
    print("resultados: "+str(len(response_json['data'])) )
    print('proximo resultados')
    print(response_json['paging'])

else:
    print("Error:", response.text)


{'ad_delivery_start_time': '2023-10-28',
 'ad_delivery_stop_time': '2023-10-29',
 'ad_snapshot_url': 'https://www.facebook.com/ads/archive/render_ad/?id=836514814873364&access_token=EAAP88z0YeOYBO31lGacDGfNB7JeGpW8ZBk198Wgnzb2MRB2ZCXvdk1720C7od21ZBVrBJZBRAkH1eo9LqZBmBejDl0SXgif83hDCjphahRlPbIXbRztZCSJuAmlPU1hkY1yAASkUYoXzRjWOruryAg4WH7Yp6dZCRaSBFLBFNT5ZARmE53E13ZCZCkbqPF3gHgtOc3jmI2f5HzZBk3Pg4IYQ0ZAxLsfhBAAPCgUHzPquVUNp',
 'id': '836514814873364',
 'page_id': '241499431662'}

{'ad_delivery_start_time': '2023-10-27',
 'ad_delivery_stop_time': '2023-10-29',
 'ad_snapshot_url': 'https://www.facebook.com/ads/archive/render_ad/?id=271341335898055&access_token=EAAP88z0YeOYBO31lGacDGfNB7JeGpW8ZBk198Wgnzb2MRB2ZCXvdk1720C7od21ZBVrBJZBRAkH1eo9LqZBmBejDl0SXgif83hDCjphahRlPbIXbRztZCSJuAmlPU1hkY1yAASkUYoXzRjWOruryAg4WH7Yp6dZCRaSBFLBFNT5ZARmE53E13ZCZCkbqPF3gHgtOc3jmI2f5HzZBk3Pg4IYQ0ZAxLsfhBAAPCgUHzPquVUNp',
 'id': '271341335898055',
 'page_id': '241499431662'}

{'ad_delivery_start_time': '2023-10-2

##Función extraer CSV candidatos

In [ ]:
def get_json_candidato(id_candidato, url_paginacion):

  # Parámetros
  params = {
      "search_page_ids": id_candidato,
      "ad_type": "POLITICAL_AND_ISSUE_ADS",
      "ad_reached_countries": ["CO"],
      "ad_delivery_date_min": "2023-08-02",
      "ad_delivery_date_max": "2023-10-29",
      "access_token": api_token
      }

  # Query
  query = 'id, ad_creative_link_descriptions,ad_creative_link_captions, ad_delivery_start_time, ad_delivery_stop_time, bylines, delivery_by_region, page_id, page_name, target_ages, target_gender, spend, currency, ad_creation_time, ad_snapshot_url, impressions, publisher_platforms, ad_creative_bodies, estimated_audience_size, demographic_distribution'

  # URL API
  url_api= "https://graph.facebook.com/v19.0/ads_archive?fields="+query

  #Solicitud get
  respuesta_json = ""

  if url_paginacion=="":
   response=requests.get(url_api,params=params)
  else:
   response=requests.get(url_paginacion)

  if response.status_code==200:
   response_json=response.json()
  else:
   raise ValueError("Error:", response.text)

  return response_json


In [ ]:
ids_candidatos=["157146724439549","209685095747841", "835710939898329", "1115184931884092", "1649204585296525", "141361183156268", "241499431662", "212483219648777", "601336570076276", "111309265404460", "110256607140478", "134134620631582", "105860962029722","204269816283669"]
df_consolidado=pd.DataFrame()

ruta="/content/drive/MyDrive/Proyecto_grado"

for i in ids_candidatos:

 url_paginacion = ""
 id_candidato = i

 df_candidato= pd.DataFrame()

 print(i)
 while True:
  json_objects=get_json_candidato(id_candidato, url_paginacion)

  df_actual= pd.DataFrame(json_objects['data'])

  df_candidato= pd.concat([df_candidato, df_actual])

  try:
     url_paginacion=json_objects["paging"]["next"]
     print("paginacion...")
  except:
     print("terminó")
     break

 nombre=df_candidato['page_name'].head(1).values[0]
 nombre = nombre.replace(" ", "_")

 df_candidato.to_csv(ruta+"/"+nombre+".csv",index=False)
 df_consolidado=pd.concat([df_consolidado,df_candidato])


df_consolidado.to_csv(ruta+"/consolidado.csv",index=False)




157146724439549
paginacion...
terminó
209685095747841
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
835710939898329
paginacion...
paginacion...
paginacion...
terminó
1115184931884092
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
1649204585296525
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
141361183156268
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
241499431662
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
p

##CSVs de All Ads


In [ ]:
# Parámetros
params = {

    "search_terms":"search_terms",
    "ad_type": "POLITICAL_AND_ISSUE_ADS",
    "ad_reached_countries": ["CO"],
    "ad_delivery_date_min":"2023-08-01",
    "ad_delivery_date_max": "2023-12-30",
    "access_token": api_token

    }

In [ ]:
# Campos de consulta
query = 'id, spend, currency, page_name, ad_creation_time, ad_delivery_start_time, ad_snapshot_url, impressions, publisher_platforms, ad_creative_bodies, estimated_audience_size, demographic_distribution'

#URL API Esta es la URL
url_api= "https://graph.facebook.com/v19.0/ads_archive"

# Realizar la solicitud GET
response = requests.get(url_api, params=params)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:

    response_json = response.json()

    for i in response_json['data']:
      print('')
      pprint(i)

    print("")
    print("resultados: "+str(len(response_json['data'])) )
    print('proximo resultados')
    print(response_json['paging'])

else:
    print("Error:", response.text)


{'ad_delivery_start_time': '2023-11-10',
 'ad_delivery_stop_time': '2023-11-11',
 'ad_snapshot_url': 'https://www.facebook.com/ads/archive/render_ad/?id=1800752177030575&access_token=EAAP88z0YeOYBO5MdXCdZALA24t9wj6YsizXK1ueZASb3NenPK3oNGVinC3K498cDuqn72ZBZAUpznL9CVa6bWTEjAjwGlm3wRArm5i8fj22nye41ZAnoyfqNLumtR5dVUbX0Fhuby5ZBwWLgDCM7ZCacXfrhPbzX5rJ6nnTyI6eAy0icc5gi8SJJtGkrp7aEoH3ZBCGvtDZBiJChr6hHJ1canhazsNuFWuxyvBjjjzN3l',
 'id': '1800752177030575',
 'page_id': '416731498371213'}

{'ad_delivery_start_time': '2023-08-02',
 'ad_delivery_stop_time': '2023-08-02',
 'ad_snapshot_url': 'https://www.facebook.com/ads/archive/render_ad/?id=1641074443064199&access_token=EAAP88z0YeOYBO5MdXCdZALA24t9wj6YsizXK1ueZASb3NenPK3oNGVinC3K498cDuqn72ZBZAUpznL9CVa6bWTEjAjwGlm3wRArm5i8fj22nye41ZAnoyfqNLumtR5dVUbX0Fhuby5ZBwWLgDCM7ZCacXfrhPbzX5rJ6nnTyI6eAy0icc5gi8SJJtGkrp7aEoH3ZBCGvtDZBiJChr6hHJ1canhazsNuFWuxyvBjjjzN3l',
 'id': '1641074443064199',
 'page_id': '100142840161003'}

{'ad_delivery_start_time': '2023-

In [ ]:
def get_json_candidato(search_terms, url_paginacion):

  # Parámetros
  params = {
      "search_terms":search_terms,
      "ad_type":"POLITICAL_AND_ISSUE_ADS",
      "ad_reached_countries": ["CO"],
      "ad_delivery_date_min": "2023-08-02",
      "ad_delivery_date_max": "2023-10-29",
      "delivery_by_region": "Antioquia",
      "access_token": api_token
      }

  # Query
  query = 'id, ad_creative_link_descriptions,ad_creative_link_captions, ad_delivery_start_time, ad_delivery_stop_time, bylines, delivery_by_region, page_id, page_name, target_ages, target_gender, spend, currency, ad_creation_time, ad_snapshot_url, impressions, publisher_platforms, ad_creative_bodies, estimated_audience_size, demographic_distribution'

  # URL API
  url_api= "https://graph.facebook.com/v19.0/ads_archive?fields="+query

  #Solicitud get
  respuesta_json = ""

  if url_paginacion=="":
   response=requests.get(url_api,params=params)
  else:
   response=requests.get(url_paginacion)

  if response.status_code==200:
   response_json=response.json()
  else:
   raise ValueError("Error:", response.text)

  return response_json


In [ ]:
searchs_terms=["Alcaldía de Medellín", "elecciones Medellín", "Alcalde de Medellín"]

df_consolidado=pd.DataFrame()

ruta="/content/drive/MyDrive/Proyecto_grado"

for i in searchs_terms:

 url_paginacion = ""
 search_terms = i

 df_terms= pd.DataFrame()

 print(i)
 while True:
  json_objects=get_json_candidato(search_terms, url_paginacion)

  df_actual= pd.DataFrame(json_objects['data'])

  df_terms= pd.concat([df_terms, df_actual])

  try:
     url_paginacion=json_objects["paging"]["next"]
     print("paginacion...")
  except:
     print("terminó")
     break

 nombre=df_terms

 nombre=df_terms['page_name'].head(1).values[0]
 nombre = nombre.replace(" ", "_")

 df_terms.to_csv(ruta+"/"+nombre+".csv",index=False)
 df_consolidado=pd.concat([df_consolidado,df_terms])

df_consolidado.to_csv(ruta+"/consolidado.csv",index=False)

Alcaldía de Medellín
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
elecciones Medellín
paginacion...

In [ ]:
searchs_terms=search_terms=["gilberto tobon","carlos ballesteros","jaime mejia","juan camilo restrepo", "luis bernardo velez", "fico","juan david valderrama","juan carlos upegui"]

df_consolidado=pd.DataFrame()

ruta="/content/drive/MyDrive/Proyecto_grado"

for i in searchs_terms:

 url_paginacion = ""
 search_terms = i

 df_terms= pd.DataFrame()

 print(i)
 while True:
  json_objects=get_json_candidato(search_terms, url_paginacion)

  df_actual= pd.DataFrame(json_objects['data'])

  df_terms= pd.concat([df_terms, df_actual])

  try:
     url_paginacion=json_objects["paging"]["next"]
     print("paginacion...")
  except:
     print("terminó")
     break

 nombre=df_terms

 nombre=df_terms['page_name'].head(1).values[0]
 nombre = nombre.replace(" ", "_")

 df_terms.to_csv(ruta+"/"+nombre+".csv",index=False)
 df_consolidado=pd.concat([df_consolidado,df_terms])

df_consolidado.to_csv(ruta+"/consolidado1.csv",index=False)

gilberto tobon
paginacion...
terminó
carlos ballesteros
paginacion...
terminó
jaime mejia
paginacion...
paginacion...
terminó
juan camilo restrepo
paginacion...
paginacion...
terminó
luis bernardo velez
paginacion...
paginacion...
terminó
fico
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...


In [ ]:
# Lista de términos de búsqueda (nombres de figuras políticas)
searchs_terms = [
    "deicy elena bermudez",
    "rodolfo correa",
    "maria paulina aguinaga",
    "liliana rendon",
    "albert corredor ",
    "yordano",
    "felipe velez",
    "federico gutierrez"
]

# Se crea un DataFrame vacío para consolidar todos los resultados
df_consolidado = pd.DataFrame()

# Ruta en Google Drive donde se guardarán los archivos .csv
ruta = "/content/drive/MyDrive/Proyecto_grado"

# Itera sobre cada término de búsqueda
for i in searchs_terms:
    url_paginacion = ""  # Inicializa la variable para manejar la paginación de resultados
    search_terms = i  # Asigna el nombre actual al término de búsqueda

    df_terms = pd.DataFrame()  # Crea un DataFrame temporal para almacenar los resultados del término actual

    print(i)  # Imprime el nombre del candidato procesado

    # Ciclo para obtener todos los resultados paginados de la API
    while True:
        json_objects = get_json_candidato(search_terms, url_paginacion)  # Llama a la función que consulta la API
        df_actual = pd.DataFrame(json_objects['data'])  # Convierte los resultados de esta página a un DataFrame
        df_terms = pd.concat([df_terms, df_actual])  # Agrega los nuevos resultados al DataFrame acumulado

        try:
            # Intenta obtener la URL de la siguiente página
            url_paginacion = json_objects["paging"]["next"]
            print("paginacion...")
        except:
            # Si no hay más páginas, termina el ciclo
            print("terminó")
            break

    # Extrae el nombre de la página del primer resultado y lo formatea para usar como nombre de archivo
    nombre = df_terms['page_name'].head(1).values[0]
    nombre = nombre.replace(" ", "_")

    # Guarda los resultados del candidato actual en un archivo CSV individual
    df_terms.to_csv(ruta + "/" + nombre + ".csv", index=False)

    # Agrega los resultados del candidato actual al DataFrame general
    df_consolidado = pd.concat([df_consolidado, df_terms])

# Guarda el archivo consolidado con todos los resultados
df_consolidado.to_csv(ruta + "/consolidado3.csv", index=False)


deicy elena bermudez
paginacion...
terminó
rodolfo correa
paginacion...
paginacion...
paginacion...
terminó
maria paulina aguinaga
paginacion...
terminó
liliana rendon
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
albert corredor 
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
yordano
paginacion...
terminó
felipe velez
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
terminó
federico gutierrez
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
paginacion...
p

##Consolidar CSV


In [ ]:
# Lista de archivos a buscar
archivos = ["consolidado", "consolidado1", "consolidado3"]
extension = 'csv'

In [ ]:
# Buscar todos los archivos con los nombres especificados y la extensión dada
todos_los_archivos = []
for archivo in archivos:
    todos_los_archivos.extend(glob.glob(f'{archivo}.{extension}'))

print(todos_los_archivos)



['consolidado.csv', 'consolidado1.csv', 'consolidado3.csv']


In [ ]:
#combina todos los archivos de la lista
combinado_csv = pd.concat([pd.read_csv(f) for f in todos_los_archivos ])
#exporta a csv
combinado_csv.to_csv( "combinado_csv.csv", index=False, encoding='utf-8-sig')

##Limpieza de datos All ADS

In [ ]:
# Ruta completa o relativa al archivo CSV
file_path = "/content/drive/MyDrive/Proyecto_grado/csv_carpetas"

#Leer datos pdf
df = pd.read_csv("combinado_csv.csv")

#Eliminar duplicados
resultado = df.drop_duplicates()

# Guardar el resultado en un nuevo archivo CSV
resultado.to_csv("Resultado.csv", index=False)


## Eliminar menos de dos registros


In [ ]:
# Ruta completa o relativa al archivo CSV
file_path = "/content/drive/MyDrive/Proyecto_grado/csv_carpetas/eliminados3.csv"

# Leer el archivo CSV en un DataFrame
df = pd.read_csv('eliminados3.csv')

# Usar groupby y transform para contar los registros por grupo
df['count'] = df.groupby('page_id')['page_id'].transform('count')

# Eliminar los grupos que tienen menos de dos registros
df = df[df['count'] >= 10]

# Eliminar la columna de conteo ya que no es necesaria
df = df.drop(columns=['count'])

# Guardar el DataFrame actualizado en el mismo archivo CSV
df.to_csv('datos.csv', index=False)

# Mostrar el DataFrame actualizado
print(df)

                    id ad_delivery_start_time ad_delivery_stop_time  \
1      353814187108904             2023-10-28            2023-10-29   
5      741570998014232             2023-10-28            2023-10-30   
6     7089407681103438             2023-10-28            2023-10-29   
7     1089909632354876             2023-10-28            2023-10-29   
8      885328363106770             2023-10-28            2023-10-29   
...                ...                    ...                   ...   
3275  3334109646735842             2023-09-28            2023-10-02   
3278   691912709498879             2023-09-08            2023-09-12   
3279   322852950197463             2023-09-08            2023-09-12   
3280  1039359350570018             2023-09-02            2023-09-12   
3283   239745772377809             2023-08-23            2023-08-30   

                   bylines                                 delivery_by_region  \
1            Santi Perdomo  [{'percentage': '0.000764', 'region': 

## Extraer archivos de anuncios

In [ ]:
# Ruta donde se encuentra el archivo CSV con los datos consolidados
ruta_csv = "/content/drive/MyDrive/Proyecto_grado/"

# Carga el archivo CSV en un DataFrame
df_consolidado = pd.read_csv(ruta_csv+'consolidado.csv')

# Inicialización de contadores
n = 1  # Contador de descargas
n_count = 0  # Contador acumulado de anuncios procesados

# Itera sobre cada fila del DataFrame
for index, row in df_consolidado.iterrows():
    # Extrae el nombre del candidato, el enlace al anuncio, y los ID's del candidato y del anuncio
    nombre_candidato = row['page_name']
    link_add = row['ad_snapshot_url']
    id_candiado = row['page_id']
    id_anuncio = row['id']

    # Filtra los registros que correspondan al candidato con ID específico (241499431662)
    if id_candiado == 241499431662:

        # Verifica si ya se ha procesado el anuncio usando una función check_file (no definida)
        if check_file(id_anuncio):
            n_count = n_count + 1  # Incrementa el contador de anuncios procesados
            continue  # Continúa con la siguiente iteración si el anuncio ya ha sido procesado

        # Si se alcanza el límite de 200 descargas, termina el ciclo
        if n == 200:
            print('limite de 200')
            break

        # Muestra el número de la descarga actual y el acumulado de anuncios procesados
        print('Descarga Actual:', n)
        print('Acumulado:', n_count)

        # Llama a la función extrac_add para procesar el enlace del anuncio
        extrac_add(link_add, nombre_candidato)
        print('####################')

        # Actualiza los contadores de descargas
        n = n + 1
        n_count = n_count + 1




ya existe el archivo: 836514814873364
####################
ya existe el archivo: 271341335898055
####################
ya existe el archivo: 3406390696340134
####################
ya existe el archivo: 778985787319053
####################
ya existe el archivo: 232006916556361
####################
ya existe el archivo: 268410085668437
####################
ya existe el archivo: 878922720490860
####################
ya existe el archivo: 349147170915179
####################
ya existe el archivo: 726807809268339
####################
ya existe el archivo: 872035550959119
####################
ya existe el archivo: 1059787358523109
####################
ya existe el archivo: 360461226331000
####################
ya existe el archivo: 141957829006597
####################
ya existe el archivo: 317379741011950
####################
ya existe el archivo: 653895690066027
####################
ya existe el archivo: 1246742329299722
####################
ya existe el archivo: 289431193964605
###############

KeyboardInterrupt: 

In [ ]:
candidato_ = ''
id_anuncio_ = ''

# Funcion principal
def extrac_add(url,nombre_candidato):

  # usar token actual
  url_dynamic = link_add.split('access_token=')
  url_dynamic = url_dynamic[0]+'access_token='+api_token

  # id del anuncio
  id_anuncio = url_dynamic.split('?id=')
  id_anuncio = id_anuncio[1].split('&')
  id_anuncio = id_anuncio[0]

  driver.get(url_dynamic)

  time.sleep(3)

  cookies = driver.find_elements(By.CLASS_NAME,"x9f619")
  for c in cookies:
    print(c.text)


  print(f"{nombre_candidato} - {url_dynamic}")

  nombre_candidato = nombre_candidato.replace(' ','_')

  global candidato_
  global id_anuncio_
  candidato_ = nombre_candidato
  id_anuncio_ = id_anuncio

  try:
    elements_bloq = driver.find_elements(By.CLASS_NAME,"x78zum5")

    if len(elements_bloq) > 4:

      if elements_bloq[4].text == 'Ver anuncio':
        print('el contenido esta bloqueado')

        btn_ver = elements_bloq[4]
        print(btn_ver.text)
        btn_ver.click() # ver el anuncio

        condiocionales(driver)  # camino normal
      else:
        condiocionales(driver)  # camino normal

    else:
      condiocionales(driver)  # camino normal


  except Exception as e: # hay un error
    print(f"Error: {e}")
    traceback.print_exc()



########################################################################
### Funcion de condicionales, camino para extraer el tipo de anuncio ###
########################################################################
def condiocionales(p_driver):

  element_video = p_driver.find_elements(By.TAG_NAME, "video")
  element_img   = p_driver.find_elements(By.TAG_NAME, "img")

  if( len(element_video) >= 1 ): # el contenido es un video
    extract_video(p_driver)

  elif( len(element_img) == 2 ): # es una imagen
    element_articulo = driver.find_elements(By.XPATH,"//a[@target='_blank']")

    if(len(element_articulo) >= 2): # con un articulo
      extract_articulo(p_driver)
    else:
      extract_img(p_driver) # extraigo la imagen

  elif( len(element_img) >= 3 ): # es una carrusel
    extract_carrusel(p_driver)



############################################
### SubFunciones para el tipo de anuncio ###
############################################
def extract_video(p_driver):
  video_elem = p_driver.find_element(By.TAG_NAME, "video")
  video_src = video_elem.get_attribute("src")
  print('video:',video_src)
  link_arr = [video_src]
  save_file(link_arr)


def extract_img(p_driver):
  img_elem = p_driver.find_elements(By.TAG_NAME,'img')
  img_dir = img_elem[1].get_attribute("src")
  print('imagen:',img_dir)
  link_arr = [img_dir]
  save_file(link_arr)

def extract_carrusel(p_driver):
  carrusel_elem = p_driver.find_elements(By.TAG_NAME,'img')
  print('carrussell')
  link_arr = []
  for i in range(1,len(carrusel_elem)):
    print(carrusel_elem[i].get_attribute("src"))
    link_arr.append( carrusel_elem[i].get_attribute("src") )
  save_file(link_arr)

def extract_articulo(p_driver):
  link_arr = []
  print('articulo')
  # extraigo imagen
  img_elem = p_driver.find_elements(By.TAG_NAME,'img')
  img_dir = img_elem[1].get_attribute("src")
  print('imagen:',img_dir)
  link_arr.append(img_dir)

  # extraigo articulo
  elements_new_tab = p_driver.find_elements(By.XPATH,"//a[@target='_blank']")
  p_driver.get(elements_new_tab[1].get_attribute("href"))
  current_url = driver.current_url
  print('link articulo:',current_url)
  link_arr.append(current_url)

  save_file(link_arr)


############################################
### SubFuncion para guaradar el anuncio ##
############################################
def save_file(link_file):
  ruta_destino = '/content/drive/MyDrive/Proyecto_grado/multimedia/'
  nombre_archivo = candidato_+'-'+id_anuncio_

  if len(link_file) > 1:  # varios archivos

    for i in range(0,len(link_file)):
      nombre_arch_iter = nombre_archivo+'-'+str(i+1)

      if 'https://scontent' not in link_file[i]: # es un link fuera de facebook
        link_string = link_file[i]
        with open( os.path.join(ruta_destino, nombre_arch_iter+".txt" ) , 'w') as archivo:
            archivo.write(link_string)
        print("Archivo guardado en", os.path.join(ruta_destino, nombre_arch_iter))
      else:
        req= requests.get( link_file[i], stream=True )
        with open(os.path.join(ruta_destino, nombre_arch_iter), "wb") as archivo:
            for chunk in req.iter_content(1024):
                archivo.write(chunk)
        print("Archivo guardado en", os.path.join(ruta_destino, nombre_arch_iter))

  else: # solo un archivo
    req= requests.get( link_file[0], stream=True )
    with open(os.path.join(ruta_destino, nombre_archivo), "wb") as archivo:
        for chunk in req.iter_content(1024):
            archivo.write(chunk)
    print("Archivo guardado en", os.path.join(ruta_destino, nombre_archivo))



# comprobar si ya esta descargado el anuncio
def check_file(p_id_anuncio):
  ruta_carpeta = '/content/drive/MyDrive/Proyecto_grado/multimedia/'
  for nombre_archivo in os.listdir(ruta_carpeta):
    if str(p_id_anuncio) in nombre_archivo:
      print('ya existe el archivo: '+ str(p_id_anuncio))
      print('####################')
      return True
  return False

